In [124]:
import os
import c3d
import glob2
import math
import numpy as np
from mayavi.mlab import *
import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.animation as animation
from matplotlib import animation, rc
from mpl_toolkits.mplot3d import Axes3D

from IPython.display import HTML, Image

%matplotlib qt

In [125]:
rc('animation', html='html5')

In [15]:
# example 3d plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = [1,2,3,4,5,6,7,8,9,10]
y = [5,6,2,3,13,4,1,2,4,8]
z = [2,3,3,3,5,7,9,11,9,10]

ax.scatter(x, y, z, c='r', marker='o') # scatter plot

ax.set_xlabel('x_axis')
ax.set_ylabel('y_axis')
ax.set_zlabel('z_axis')

plt.show()

In [16]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y, Z = [1,2,3,4,5,6,7,8,9,10],[5,6,2,3,13,4,1,2,4,8],[2,3,3,3,5,7,9,11,9,10]
ax.plot(X, Y, Z, label='parametric curve')

plt.show()

In [17]:
mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')
theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
z = np.linspace(-2, 2, 100)
r = z**2 + 1
x = r * np.sin(theta)
y = r * np.cos(theta)
ax.plot(x, y, z, label='parametric curve')
ax.legend()

plt.show()

In [21]:
# try to read c3d files
cd3data_path = os.path.join('data', 'motion_data', 'files_motions_589')
all_files = glob2.glob(cd3data_path+'/**/*[0-9].c3d', recursive=True)
for files in all_files: 
    print(files)

# read c3d file
reader = c3d.Reader(open(all_files[0], 'rb'))

data/motion_data/files_motions_589/pour02.c3d
data/motion_data/files_motions_589/pour04.c3d
data/motion_data/files_motions_589/pour03.c3d
data/motion_data/files_motions_589/pour01.c3d
data/motion_data/files_motions_589/pour05.c3d
data/motion_data/files_motions_589/pour06.c3d


### See information about marker points [here](https://motion-database.humanoids.kit.edu/marker_set/)

In [22]:
# reader.point_used
point_labels = reader.point_labels
# print(type(point_labels))
reader.point_labels

[u'small_cup:sc_01               ',
 u'small_cup:sc_02               ',
 u'small_cup:sc_03               ',
 u'small_cup:sc_04               ',
 u'bottle:shampoo_01             ',
 u'bottle:shampoo_02             ',
 u'bottle:shampoo_03             ',
 u'bottle:shampoo_04             ',
 u'Table:table_1                 ',
 u'Table:table_2                 ',
 u'Table:table_3                 ',
 u'Table:table_4                 ',
 u'subjX:RPSI                    ',
 u'subjX:LPSI                    ',
 u'subjX:L3                      ',
 u'subjX:STRN                    ',
 u'subjX:T10                     ',
 u'subjX:C7                      ',
 u'subjX:CLAV                    ',
 u'subjX:RBAK                    ',
 u'subjX:LBAK                    ',
 u'subjX:LSHO                    ',
 u'subjX:LUPA                    ',
 u'subjX:LAEL                    ',
 u'subjX:LAOL                    ',
 u'subjX:LWTS                    ',
 u'subjX:LWPS                    ',
 u'subjX:LFRA               

In [23]:
# the marker points that we need
# left_arm_markers = ['LSHO', 'LUPA', 'LAEL', 'LAOL', 'LWTS', 'LWPS', 'LIFD']
# right_arm_markers = ['RSHO', 'RUPA', 'RAEL', 'RAOL', 'RWTS', 'RWPS', 'RIFD']
left_arm_points_ids = list(range(21,31))
right_arm_points_ids = list(range(31,41))
print(len(right_arm_points_ids))

10


In [103]:
right_arm_points = []
for point_id in right_arm_points_ids:
    point_marker = []
    for i, points, analog in reader.read_frames():
        point_marker.append(points[point_id][:3])
    right_arm_points.append(point_marker)

In [104]:
points_array = np.array(right_arm_points)
print(np.shape(points_array))

(10, 831, 3)


In [26]:
# visualize a frame
frame = points_array[:,0,:]
x_data = frame[:,0]
y_data = frame[:,1]
z_data = frame[:,2]

In [64]:
# plot joint positions
fig = plt.figure()
aax = fig.gca(projection='3d')

aax.scatter(x_data, y_data, z_data)
# aax.plot(x_data, y_data, z_data)

aax.set_xlabel('x_axis')
aax.set_ylabel('y_axis')
aax.set_zlabel('z_axis')

aax.azim = 180
aax.elev = 10
aax.autoscale()
# aax.legend()

# %matplotlib qt
plt.show()

### we are going to try to make animations of the arm movement

#### Example:

In [65]:
"""
============
3D animation
============

A simple example of an animated plot... In 3D!
"""
# import numpy as np
# import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
# import matplotlib.animation as animation


def Gen_RandLine(length, dims=2):
    """
    Create a line using a random walk algorithm

    length is the number of points for the line.
    dims is the number of dimensions the line has.
    """
    lineData = np.empty((dims, length))
    lineData[:, 0] = np.random.rand(dims)
    for index in range(1, length):
        # scaling the random numbers by 0.1 so
        # movement is small compared to position.
        # subtraction by 0.5 is to change the range to [-0.5, 0.5]
        # to allow a line to move backwards.
        step = ((np.random.rand(dims) - 0.5) * 0.1)
        lineData[:, index] = lineData[:, index - 1] + step

    return lineData


def update_lines(num, dataLines, lines):
    for line, data in zip(lines, dataLines):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:2, :num])
        line.set_3d_properties(data[2, :num])
    return lines

# Attaching 3D axis to the figure
fig = plt.figure()
ax = p3.Axes3D(fig)

# Fifty lines of random 3-D lines
data = [Gen_RandLine(25, 3) for index in range(50)]

# Creating fifty line objects.
# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]

# Setting the axes properties
ax.set_xlim3d([0.0, 1.0])
ax.set_xlabel('X')

ax.set_ylim3d([0.0, 1.0])
ax.set_ylabel('Y')

ax.set_zlim3d([0.0, 1.0])
ax.set_zlabel('Z')

ax.set_title('3D Test')

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update_lines, 25, fargs=(data, lines),
                                   interval=50, blit=False)

plt.show()


In [29]:
np.shape([Gen_RandLine(25, 3) for index in range(50)])

(50, 3, 25)

In [30]:
frame = points_array[:,0,:]
x_data = frame[:,0]
y_data = frame[:,1]
z_data = frame[:,2]

print(np.shape(x_data))

(10,)


In [53]:
np.shape(np.transpose(points_array[:,:50,:], (1,2,0)))

(50, 3, 10)

In [74]:
def frame_gen():
    total_frames = np.shape(points_array)[0]
    t = 0
    while t < total_frames:
        frame = points_array[:,t,:]
        x_data = frame[:,0]
        y_data = frame[:,1]
        z_data = frame[:,2]
        t += 1
        yield x_data, y_data, z_data

def update_lines(num, dataLines, lines):
    for line, data in zip(lines, dataLines):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:4, :num])
        line.set_3d_properties(data[2, :num])
    return lines

# Attaching 3D axis to the figure
fig = plt.figure()
# c3d_plot = p3.Axes3D(fig)

c3d_plot = fig.gca(projection='3d')



# samples of 3d points
data = np.transpose(points_array[:,:50,:], (1,2,0))

# Setting the axes properties
c3d_plot.set_xlim3d([0.0, 200.0])
c3d_plot.set_xlabel('X')

c3d_plot.set_ylim3d([200.0, 600.0])
c3d_plot.set_ylabel('Y')

c3d_plot.set_zlim3d([500.0, 1500.0])
c3d_plot.set_zlabel('Z')

c3d_plot.azim = 180
c3d_plot.elev = 10

c3d_plot.set_title('C3D Arm Points 3D')

# Creating fifty line objects.
# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [c3d_plot.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]
# lines = [c3d_plot.scatter3D(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1]) for dat in data]

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update_lines, 10, fargs=(data, lines),
                                   interval=50, blit=False)

plt.show()

In [90]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import pandas as pd


a = np.random.rand(2000, 3)*10
t = np.array([np.ones(100)*i for i in range(20)]).flatten()
df = pd.DataFrame({"time": t ,"x" : a[:,0], "y" : a[:,1], "z" : a[:,2]})

def update_graph(num):
    data=df[df['time']==num]
    graph._offsets3d = (data.x, data.y, data.z)
    title.set_text('3D Test, time={}'.format(num))


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
title = ax.set_title('3D Test')

data=df[df['time']==0]
graph = ax.scatter(data.x, data.y, data.z)

ani = matplotlib.animation.FuncAnimation(fig, update_graph, 19, 
                               interval=40, blit=False)

plt.show()

In [102]:
data.shape

(100, 4)

In [106]:
np.shape(points_array[:,0,:][:,0])


(10,)

In [123]:
def update_graph(num):
    frame = points_array[:,num,:]
    x_data = frame[:,0]
    y_data = frame[:,1]
    z_data = frame[:,2]
    graph._offsets3d = (x_data, y_data, z_data)
    title.set_text('Right Hand Demo, time step={}'.format(num))


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Setting the axes properties
ax.set_xlim3d([-500.0, 200.0])
ax.set_xlabel('X')

ax.set_ylim3d([-100.0, 600.0])
ax.set_ylabel('Y')

ax.set_zlim3d([500.0, 1500.0])
ax.set_zlabel('Z')

ax.azim = 180
ax.elev = 10

title = ax.set_title('3D Test')

# plot initial data
x_data = points_array[:,0,:][:,0]
y_data = points_array[:,0,:][:,1]
z_data = points_array[:,0,:][:,2]

graph = ax.scatter(x_data, y_data, z_data)

ani = matplotlib.animation.FuncAnimation(fig, update_graph, 831,
                               interval=1, blit=False)

# can save as mp4
# ani.save('basic_animation.mp4', fps=30)
ani.save('data/media_outputs/basic_animation.gif', writer='imagemagick', fps=30)

plt.show()

In [128]:
# display image or animation through here:
# Image(url='data/media_outputs/basic_animation.gif')